# Compiler Construction Project

# 1): Scanner Program Code

# In Scanner we are taking user input from read.py file

In [1]:
import re
import pandas as pd
from IPython.display import display, HTML

# Token class to represent individual tokens
class Token:
    def __init__(self, type, value, position):
        self.type = type
        self.value = value
        self.position = position

    def __repr__(self):
        return f'({self.type}, {self.value})'

# Enum class for token types
class TokenType:
    DS = 'DS'
    T = 'T'
    B = 'B'
    V = 'V'
    L = 'L'
    D = 'D'
    X = 'X'
    Y = 'Y'
    S = 'S'
    O = 'O'
    W = 'W'
    N = 'N'
    C = 'C'
    BL = 'BL'
    Identifier = 'Identifier'
    Number = 'Number'
    Operator = 'Operator'
    Separator = 'Separator'
    Keyword = 'Keyword'
    Comment = 'Comment'

# GeneratedScanner class with regular expressions for the CFG
class GeneratedScanner:
    DS = re.compile(r'^TBVO$')
    T = re.compile(r'^(int|float|char|string|bool)$')
    B = re.compile(r'^(bB|b)$')
    V = re.compile(r'^XY$')
    L = re.compile(r'^(A|B|\.\.\.|Z|a|b|\.\.\.|z|X|Y|c)$')  # Added patterns for X, Y, and c
    D = re.compile(r'^(0|1|2|\.\.\.|9)$')
    X = re.compile(r'^X$')
    Y = re.compile(r'^Y$')
    S = re.compile(r'^(aS|bS|\^|,|\;)$')  # Updated S pattern to include semicolon
    O = re.compile(r'^(i|==)$')  # Updated O pattern
    W = re.compile(r'^(if|else|while|int|float|char|string|bool)$')
    N = re.compile(r'^\d+(\.\d+)?$')
    C = re.compile(r'^(0|1|\.\.\.|9|A|\.\.\.|a|\.\.\.|\+|\-|\(|\)|\.\.\.)$')  # Added pattern for )
    BL = re.compile(r'^(True|False)$')


# Scanner class for tokenizing code
class Scanner:
    # Define regular expressions for the new CFG
    identifier_regex = re.compile(r'^[a-zA-Z_]\w*$')
    number_regex = re.compile(r'^\d+(\.\d+)?$')  # Modified for floating-point numbers
    operators = ['==', '<=', '>=', '!=', '=', '<', '>']
    separators = [',', '(', ')']
    keywords = ['if', 'else', 'while', 'int', 'float', 'char', 'string', 'bool']  # Added missing data types

    def __init__(self):
        self.tokens = []
        self.code_buffer = ''
        self.inside_if_else = 0  # Track the nesting level of if-else blocks

    # Display token buffer in HTML format
    def display_buffer(self):
        html_buffer = '<table style="width:100%; border: 1px solid black;">'
        html_buffer += '<tr><th style="border: 1px solid black; text-align: left;">Token Type</th>'
        html_buffer += '<th style="border: 1px solid black; text-align: left;">Value</th>'
        html_buffer += '<th style="border: 1px solid black; text-align: left;">Memory Address</th></tr>'

        for token in self.tokens:
            type_style = 'color: green;' if token.type == TokenType.Identifier else 'color: red;'
            value_style = 'color: blue;' if token.type == TokenType.Number else 'color: black;'
            address_style = 'color: purple;'

            html_buffer += f'<tr><td style="border: 1px solid black; {type_style}; text-align: left;">{token.type}</td>'
            html_buffer += f'<td style="border: 1px solid black; {value_style}; text-align: left;">{token.value}</td>'
            html_buffer += f'<td style="border: 1px solid black; {address_style}; text-align: left;">{hex(id(token))}</td></tr>'

        html_buffer += '</table>'
        return HTML(html_buffer)

    # Tokenize the given line of code
    def tokenize(self, line):
        self.code_buffer += line
        matches = re.finditer(r'(DS|T|B|V|L|D|X|Y|S|O|W|N|C|BL|[a-zA-Z_]\w*|\d+(\.\d+)?|==|<=|>=|!=|[<>]=|[();]+|\/\/.*)', line)
        current_position = 0

        for match in matches:
            word = match.group()
            if word.startswith('//'):
                self.tokens.append(Token(TokenType.Comment, word[2:], current_position))
            else:
                self.tokens.append(self.get_token(word, current_position))
            current_position += len(word)

    # Get token based on the generated CFG
    def get_token(self, word, position):
        if GeneratedScanner.DS.match(word):
            return Token(TokenType.DS, word, position)
        elif GeneratedScanner.T.match(word):
            return Token(TokenType.T, word, position)
        elif GeneratedScanner.B.match(word):
            return Token(TokenType.B, word, position)
        elif GeneratedScanner.V.match(word):
            return Token(TokenType.V, word, position)
        elif GeneratedScanner.L.match(word):
            return Token(TokenType.L, word, position)
        elif GeneratedScanner.D.match(word):
            return Token(TokenType.D, word, position)
        elif GeneratedScanner.X.match(word):
            return Token(TokenType.X, word, position)
        elif GeneratedScanner.Y.match(word):
            return Token(TokenType.Y, word, position)
        elif GeneratedScanner.S.match(word):
            return Token(TokenType.S, word, position)
        elif GeneratedScanner.O.match(word):
            return Token(TokenType.O, word, position)
        elif GeneratedScanner.W.match(word):
            return Token(TokenType.W, word, position)
        elif GeneratedScanner.N.match(word):
            return Token(TokenType.N, word, position)
        elif GeneratedScanner.C.match(word):
            return Token(TokenType.C, word, position)
        elif GeneratedScanner.BL.match(word):
            return Token(TokenType.BL, word, position)
        elif self.identifier_regex.match(word):
            return Token(TokenType.Identifier, word, position)
        elif self.number_regex.match(word):
            return Token(TokenType.Number, word, position)
        elif word in self.operators:
            return Token(TokenType.Operator, word, position)
        elif word == ';':
            return Token(TokenType.Separator, word, position)
        elif any(word.startswith(op) for op in self.operators):
            operator = next(op for op in self.operators if word.startswith(op))
            return Token(TokenType.Operator, operator, position)
        elif word in self.separators:
            return Token(TokenType.Separator, word, position)
        elif word in self.keywords:
            return Token(TokenType.Keyword, word, position)
        else:
            print(f'Warning: Unknown token - {word}')
            return Token(TokenType.Comment, f'Unknown token - {word}', position)


# Display token table in HTML format
def display_token_table(tokens):
    data = {'Index': range(len(tokens)),
            'Token Type': [token.type for token in tokens],
            'Value': [token.value if hasattr(token, "value") else "" for token in tokens]}
    df = pd.DataFrame(data)
    df_html = df.style.set_table_styles([
        {'selector': 'th', 'props': [('background-color', 'lightgrey'), ('border', '1px solid black'), ('text-align', 'left')]},
        {'selector': 'td', 'props': [('border', '1px solid black'), ('text-align', 'left')]},
    ]).render()
    return HTML(df_html)

# Display CFG
def display_cfg():
    cfg = '''
    DS -> TBVO
    T -> int | float | char | string | bool
    B -> bB | b
    V -> XY
    L -> A | B | ... | Z | a | b | ... | z | X | Y | c
    D -> 0 | 1 | 2 | ... | 9
    X -> X
    Y -> Y
    S -> aS | bS | ^ | , | ;
    O -> i | ==
    W -> if | else | while | int | float | char | string | bool
    N -> \d+(\.\d+)?
    C -> 0 | 1 | ... | 9 | A | ... | a | ... | + | - | ( | ) | ...
    BL -> True | False
    '''

    print("\nContext-Free Grammar (CFG):\n")
    print(cfg)

# Validate code against CFG
def validate_code(declarative_code):
    # Initialize the scanner
    scanner = Scanner()

    # Tokenize the declarative code
    for line in declarative_code.split('\n'):
        scanner.tokenize(line)

    # Validate tokens against CFG
    valid_tokens = []
    for token in scanner.tokens:
        if token.type == TokenType.Comment:
            continue  # Ignore comments for validation

        # Validate token based on CFG
        validation_result = False

        if scanner.inside_if_else:
            # Inside an if-else block, only validate specific tokens
            if token.type in (TokenType.Keyword, TokenType.Separator, TokenType.Operator, TokenType.Number):
                validation_result = True
        else:
            # Outside of an if-else block, validate according to the CFG
            if token.type == TokenType.Keyword:
                validation_result = token.value in Scanner.keywords
            else:
                for pattern in GeneratedScanner.__dict__.values():
                    if isinstance(pattern, re.Pattern) and pattern.match(token.value):
                        validation_result = True
                        break

        valid_tokens.append((token, validation_result))

        # Update inside_if_else flag
        if token.type == TokenType.Keyword and token.value == 'if':
            scanner.inside_if_else += 1
        elif token.type == TokenType.Separator and token.value == '}':
            scanner.inside_if_else -= 1

    # Check if all tokens are valid
    scanner_valid = all(valid for token, valid in valid_tokens)
    return valid_tokens, scanner_valid

# Main execution block
if __name__ == '__main__':
    scanner = Scanner()

    # Replace this with your code or provide your code directly here
    declarative_code = '''
    int a;
    float b;
    XY c;
    if (a == b) {
        a = 10;
    } else {
        b = 20;
    }
    '''

    # Tokenize and validate the provided code
    for line in declarative_code.split('\n'):
        scanner.tokenize(line)

    # Display CFG
    display_cfg()
    
    # Display buffer and lexeme table
    display(HTML("\nBuffer:"))
    display(scanner.display_buffer())

    display(HTML("\nLexeme Table:"))
    display(display_token_table(scanner.tokens))

    # Validate code against CFG
    validation_result, scanner_valid = validate_code(declarative_code)

    print("\nValidation Result:")
    for token, valid in validation_result:
        print(f'Token: {token}, Valid: {valid}')

    print("\nScanner is valid:", scanner_valid)


Context-Free Grammar (CFG):


    DS -> TBVO
    T -> int | float | char | string | bool
    B -> bB | b
    V -> XY
    L -> A | B | ... | Z | a | b | ... | z | X | Y | c
    D -> 0 | 1 | 2 | ... | 9
    X -> X
    Y -> Y
    S -> aS | bS | ^ | , | ;
    O -> i | ==
    W -> if | else | while | int | float | char | string | bool
    N -> \d+(\.\d+)?
    C -> 0 | 1 | ... | 9 | A | ... | a | ... | + | - | ( | ) | ...
    BL -> True | False
    


Token Type,Value,Memory Address
T,int,0x27bc347a370
L,a,0x27bc1468070
S,;,0x27bc1468100
T,float,0x27bc1468250
B,b,0x27bc14681c0
S,;,0x27bc1468310
L,X,0x27bc1468190
L,Y,0x27bc14681f0
L,c,0x27bc1468160
S,;,0x27bc1458d60


,Index,Token Type,Value
0,0,T,int
1,1,L,a
2,2,S,;
3,3,T,float
4,4,B,b
5,5,S,;
6,6,L,X
7,7,L,Y
8,8,L,c
9,9,S,;



Validation Result:
Token: (T, int), Valid: True
Token: (L, a), Valid: True
Token: (S, ;), Valid: True
Token: (T, float), Valid: True
Token: (B, b), Valid: True
Token: (S, ;), Valid: True
Token: (L, X), Valid: True
Token: (L, Y), Valid: True
Token: (L, c), Valid: True
Token: (S, ;), Valid: True
Token: (W, if), Valid: True
Token: (C, (), Valid: True
Token: (L, a), Valid: True
Token: (O, ==), Valid: True
Token: (B, b), Valid: True
Token: (C, )), Valid: True
Token: (L, a), Valid: True
Token: (N, 10), Valid: True
Token: (S, ;), Valid: True
Token: (W, else), Valid: True
Token: (B, b), Valid: True
Token: (N, 20), Valid: True
Token: (S, ;), Valid: True

Scanner is valid: True


# 2: Parser Program Code

# LL(1) Parser

# Input String: 

(i+i)*i$
or i+i*i$ 
and may be more


In [1]:
from collections import OrderedDict
from IPython.display import display, HTML

def isterminal(char):
    if(char.isupper() or char == "`"):
        return False
    else:
        return True

def insert(grammar, lhs, rhs):
    if(lhs in grammar and rhs not in grammar[lhs] and grammar[lhs] != "null"):
        grammar[lhs].append(rhs)
    elif(lhs not in grammar or grammar[lhs] == "null"):
        grammar[lhs] = [rhs]
    return grammar

def first(lhs, grammar, grammar_first):
    rhs = grammar[lhs]
    for i in rhs:
        k = 0
        flag = 0
        current = []
        confirm = 0
        flog = 0
        if(lhs in grammar and "`" in grammar_first[lhs]):
            flog = 1
        while(1):    
            check = []
            if(k >= len(i)):
                if(len(current) == 0 or flag == 1 or confirm == k or flog == 1):
                    grammar_first = insert(grammar_first, lhs, "`")
                break                
            if(i[k].isupper()):
                if(grammar_first[i[k]] == "null"):
                    grammar_first = first(i[k], grammar, grammar_first)
                for j in grammar_first[i[k]]:
                    grammar_first = insert(grammar_first, lhs, j)
                    check.append(j)
            else:
                grammar_first = insert(grammar_first, lhs, i[k])
                check.append(i[k])
            if(i[k] == "`"):
                flag = 1
            current.extend(check)
            if("`" not in check):
                if(flog == 1):
                    grammar_first = insert(grammar_first, lhs, "`")
                break
            else:
                confirm += 1
                k += 1
                grammar_first[lhs].remove("`")
    return(grammar_first)

def rec_follow(k, next_i, grammar_follow, i, grammar, start, grammar_first, lhs, recursion_steps):
    if(len(k) == next_i):
        if(grammar_follow[i] == "null"):
            grammar_follow, recursion_steps = follow(i, grammar, grammar_follow, start, recursion_steps)
        for q in grammar_follow[i]:
            grammar_follow = insert(grammar_follow, lhs, q)
            recursion_steps.append(f"{lhs} -> {k} [Follow({i})]")
    else:
        if(k[next_i].isupper()):
            for q in grammar_first[k[next_i]]:
                if(q == "`"):
                    grammar_follow, recursion_steps = rec_follow(k, next_i + 1, grammar_follow, i, grammar, start, grammar_first, lhs, recursion_steps)        
                else:
                    grammar_follow = insert(grammar_follow, lhs, q)
                    recursion_steps.append(f"{lhs} -> {k} [First({k[next_i]})]")
        else:
            grammar_follow = insert(grammar_follow, lhs, k[next_i])
            recursion_steps.append(f"{lhs} -> {k} [Terminal({k[next_i]})]")

    return(grammar_follow, recursion_steps)

def follow(lhs, grammar, grammar_follow, start, recursion_steps=None):
    if recursion_steps is None:
        recursion_steps = []

    for i in grammar:
        j = grammar[i]
        for k in j:
            if(lhs in k):
                next_i = k.index(lhs) + 1
                grammar_follow, recursion_steps = rec_follow(k, next_i, grammar_follow, i, grammar, start, grammar_first, lhs, recursion_steps)

    if(lhs == start):
        grammar_follow = insert(grammar_follow, lhs, "$")
        recursion_steps.append(f"{lhs} is the start symbol, add $ to Follow({lhs})")

    return(grammar_follow, recursion_steps)

def show_dict(dictionary):
    table_html = "<table><tr><th>Non-terminal</th><th>Elements</th></tr>"
    for key in dictionary.keys():
        table_html += f"<tr><td>{key}</td><td>"
        for item in dictionary[key]:
            if(item == "`"):
                table_html += "Epsilon, "
            else:
                table_html += f"{item}, "
        table_html += "</td></tr>"
    table_html += "</table>"
    display(HTML(table_html))

def get_rule(non_terminal, terminal, grammar, grammar_first):
    for rhs in grammar[non_terminal]:
        for rule in rhs:
            if(rule == terminal):
                string = non_terminal + "~" + rhs
                return string
            elif(rule.isupper() and terminal in grammar_first[rule]):
                string = non_terminal + "~" + rhs
                return string

def generate_parse_table(terminals, non_terminals, grammar, grammar_first, grammar_follow):
    parse_table = [[""] * len(terminals) for i in range(len(non_terminals))]
    for non_terminal in non_terminals:
        for terminal in terminals:
            if terminal in grammar_first[non_terminal]:
                rule = get_rule(non_terminal, terminal, grammar, grammar_first)
            elif("`" in grammar_first[non_terminal] and terminal in grammar_follow[non_terminal]):
                rule = non_terminal + "~`"
            elif(terminal in grammar_follow[non_terminal]):
                rule = "Sync"
            else:
                rule = ""
            parse_table[non_terminals.index(non_terminal)][terminals.index(terminal)] = rule
    return(parse_table)

def display_parse_table(parse_table, terminals, non_terminals):
    table_html = "<table><tr><th></th>"

    for terminal in terminals:
        table_html += f"<th>{terminal}</th>"
    table_html += "</tr>"

    for i, non_terminal in enumerate(non_terminals):
        table_html += f"<tr><td>{non_terminal}</td>"
        for j, terminal in enumerate(terminals):
            table_html += f"<td>{parse_table[i][j]}</td>"
        table_html += "</tr>"

    table_html += "</table>"
    display(HTML(table_html))

def display_parsing_expr(expr):
    table_html = "<table><tr><th>Matched</th><th>Stack</th><th>Input</th><th>Action</th></tr>"

    stack = ["$"]
    stack.insert(0, non_terminals[0])
    matched = "-"
    
    while(True):
        action = "-"

        if(stack[0] == expr[0] and stack[0] == "$"):
            break

        elif(stack[0] == expr[0]):
            if(matched == "-"):
                matched = expr[0]
            else:
                matched = matched + expr[0]
            action = "Matched " + expr[0]
            expr = expr[1:]
            stack.pop(0)

        else:
            action = parse_table[non_terminals.index(stack[0])][terminals.index(expr[0])]
            stack.pop(0)
            i = 0
            for item in action[2:]:
                if(item != "`"):
                    stack.insert(i, item)
                i += 1

        table_html += f"<tr><td>{matched}</td>"
        table_html += "<td>"
        for i in stack:
            table_html += f"{i}"
        table_html += "</td>"
        table_html += f"<td>{expr}</td>"
        table_html += f"<td>{action}</td></tr>"

    table_html += "</table>"
    display(HTML(table_html))

# Main Driver
grammar = OrderedDict()
grammar_first = OrderedDict()
grammar_follow = OrderedDict()

f = open('grammar.txt')
for i in f:
    i = i.replace("\n", "")
    lhs = ""
    rhs = ""
    flag = 1
    for j in i:
        if(j == "~"):
            flag = (flag + 1) % 2
            continue
        if(flag == 1):
            lhs += j
        else:
            rhs += j
    grammar = insert(grammar, lhs, rhs)
    grammar_first[lhs] = "null"
    grammar_follow[lhs] = "null"

expr = input("Enter the expression ending with $ : ")

print("Grammar\n")
show_dict(grammar)

for lhs in grammar:
    if(grammar_first[lhs] == "null"):
        grammar_first = first(lhs, grammar, grammar_first)

print("\n")
print("First\n")
show_dict(grammar_first)

start = list(grammar.keys())[0]

print("\n")
print("Recursion Steps")
for lhs in grammar:
    if(grammar_follow[lhs] == "null"):
        grammar_follow, recursion_steps = follow(lhs, grammar, grammar_follow, start)
        print("")
        print("\n".join(recursion_steps))  # Display recursion steps for Follow set

print("\n")
print("Follow\n")
show_dict(grammar_follow)

non_terminals = list(grammar.keys())
terminals = []
for i in grammar:
    for rule in grammar[i]:
        for char in rule:
            if(isterminal(char) and char not in terminals):
                terminals.append(char)

terminals.append("$")

print("\n\t\t\t\t\t\t\tParse Table\n")
parse_table = generate_parse_table(terminals, non_terminals, grammar, grammar_first, grammar_follow)
display_parse_table(parse_table, terminals, non_terminals)

print("\n")
print("\t\t\t\t\t\t\tParsing Expression\n")
display_parsing_expr(expr)
print("\n\n\n\n\n\n")

Enter the expression ending with $ : i+i*i$
Grammar



Non-terminal,Elements
E,"TL,"
L,"+TL, Epsilon,"
T,"FK,"
K,"*FK, Epsilon,"
F,"i, (E),"




First



Non-terminal,Elements
E,"i, (,"
L,"+, Epsilon,"
T,"i, (,"
K,"*, Epsilon,"
F,"i, (,"




Recursion Steps

E -> (E) [Terminal())]
E is the start symbol, add $ to Follow(E)

L -> TL [Follow(E)]
L -> TL [Follow(E)]
L -> +TL [Follow(L)]
L -> +TL [Follow(L)]

T -> TL [First(L)]
T -> TL [Follow(E)]
T -> TL [Follow(E)]
T -> +TL [First(L)]
T -> +TL [Follow(L)]
T -> +TL [Follow(L)]

K -> FK [Follow(T)]
K -> FK [Follow(T)]
K -> FK [Follow(T)]
K -> *FK [Follow(K)]
K -> *FK [Follow(K)]
K -> *FK [Follow(K)]

F -> FK [First(K)]
F -> FK [Follow(T)]
F -> FK [Follow(T)]
F -> FK [Follow(T)]
F -> *FK [First(K)]
F -> *FK [Follow(K)]
F -> *FK [Follow(K)]
F -> *FK [Follow(K)]


Follow



Non-terminal,Elements
E,"), $,"
L,"), $,"
T,"+, ), $,"
K,"+, ), $,"
F,"*, +, ), $,"



							Parse Table



,+,*,i,(,),$
E,,,E~TL,E~TL,Sync,Sync
L,L~+TL,,,,L~`,L~`
T,Sync,,T~FK,T~FK,Sync,Sync
K,K~`,K~*FK,,,K~`,K~`
F,Sync,Sync,F~i,F~(E),Sync,Sync




							Parsing Expression



Matched,Stack,Input,Action
-,TL$,i+i*i$,E~TL
-,FKL$,i+i*i$,T~FK
-,iKL$,i+i*i$,F~i
i,KL$,+i*i$,Matched i
i,L$,+i*i$,K~`
i,+TL$,+i*i$,L~+TL
i+,TL$,i*i$,Matched +
i+,FKL$,i*i$,T~FK
i+,iKL$,i*i$,F~i
i+i,KL$,*i$,Matched i
